In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

23688


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
23683,2021-05-29,47,沖縄県,335,16631,0,148
23684,2021-05-30,47,沖縄県,271,16902,0,148
23685,2021-05-31,47,沖縄県,142,17044,2,150
23686,2021-06-01,47,沖縄県,223,17267,1,151
23687,2021-06-02,47,沖縄県,297,17564,1,152


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,11088,8473,138
43,香川県,18648,2018,26
44,高知県,19656,1520,20
45,鳥取県,15624,466,2
46,鹿児島県,23184,3384,30


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
23681,2021-05-27,47,沖縄県,240,15983,0,148,1496.0
23682,2021-05-28,47,沖縄県,313,16296,0,148,1602.0
23683,2021-05-29,47,沖縄県,335,16631,0,148,1706.0
23684,2021-05-30,47,沖縄県,271,16902,0,148,1821.0
23685,2021-05-31,47,沖縄県,142,17044,2,150,1859.0
23686,2021-06-01,47,沖縄県,223,17267,1,151,1826.0
23687,2021-06-02,47,沖縄県,297,17564,1,152,1821.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref.tail()

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均
23683,2021-05-29,47,沖縄県,335,16631,0,148,1706.0,0.070288,0.164625
23684,2021-05-30,47,沖縄県,271,16902,0,148,1821.0,-0.191045,0.183715
23685,2021-05-31,47,沖縄県,142,17044,2,150,1859.0,-0.476015,0.163332
23686,2021-06-01,47,沖縄県,223,17267,1,151,1826.0,0.570423,0.036030
23687,2021-06-02,47,沖縄県,297,17564,1,152,1821.0,0.331839,0.057765


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["都道府県名","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均"]]
latest.head()

,都道府県名,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均
503,北海道,2573.0,0.248031,-0.057105
1007,青森県,120.0,-0.062500,1.315065
1511,岩手県,85.0,1.833333,0.217478
2015,宮城県,117.0,0.000000,0.395505
2519,秋田県,5.0,-1.000000,NaN


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均
0,三重県,12096,4958,107,183.0,0.045455,-0.013708
1,京都府,13104,15917,215,325.0,0.937500,0.102367
2,佐賀県,20664,2513,19,63.0,-0.571429,0.143707
3,兵庫県,14112,39920,1202,697.0,-0.071429,0.268710
4,北海道,504,38530,1138,2573.0,0.248031,-0.057105


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,人口
0,三重県,12096,4958,107,183.0,0.045455,-0.013708,1813859
1,京都府,13104,15917,215,325.0,0.937500,0.102367,2545899
2,佐賀県,20664,2513,19,63.0,-0.571429,0.143707,823810
3,兵庫県,14112,39920,1202,697.0,-0.071429,0.268710,5549568
4,北海道,504,38530,1138,2573.0,0.248031,-0.057105,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,504,38530,1138,2573.0,0.248031,-0.057105,5267762,73.143016,731.430160,48.844272
41,青森県,1008,2383,30,120.0,-0.062500,1.315065,1275783,18.678725,186.787251,9.405988
19,岩手県,1512,1465,45,85.0,1.833333,0.217478,1235517,11.857384,118.573844,6.879711
11,宮城県,2016,8894,83,117.0,0.000000,0.395505,2292385,38.798020,387.980204,5.103855
36,秋田県,2520,758,14,5.0,-1.000000,NaN,985416,7.692183,76.921828,0.507400
15,山形県,3024,1971,42,50.0,1.333333,0.002706,1082296,18.211284,182.112842,4.619808
35,福島県,3528,4602,144,93.0,1.800000,0.089002,1881981,24.452957,244.529568,4.941601
38,茨城県,4032,9797,151,254.0,0.520000,-0.003524,2921436,33.534878,335.348781,8.694354
27,栃木県,4536,6402,78,219.0,-0.181818,0.071658,1965516,32.571600,325.715995,11.142112
37,群馬県,5040,7861,139,159.0,0.352941,-0.024911,1969439,39.914920,399.149199,8.073365


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      15942.191489
std       29297.801201
min         466.000000
25%        2448.000000
50%        4823.000000
75%       10303.000000
max      161913.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,6552,161913,2075,3503.0,0.033970,-0.006695,13834925,117.032076,1170.320764,25.319978
1,大阪府,13608,100133,2363,1524.0,0.059701,0.019585,8849635,113.149299,1131.492994,17.221049
2,神奈川県,7056,61892,896,1494.0,0.371069,0.023027,9209442,67.204940,672.049403,16.222481
3,愛知県,11592,47882,816,2165.0,-0.059016,0.023445,7575530,63.206139,632.061387,28.578859
4,埼玉県,5544,44020,792,825.0,0.157895,0.038890,7390054,59.566547,595.665471,11.163653
5,兵庫県,14112,39920,1202,697.0,-0.071429,0.268710,5549568,71.933527,719.335271,12.559536
6,北海道,504,38530,1138,2573.0,0.248031,-0.057105,5267762,73.143016,731.430160,48.844272
7,千葉県,6048,37303,681,691.0,0.730159,0.019925,6319772,59.025864,590.258636,10.933939
8,福岡県,20160,34300,465,974.0,0.432099,-0.026040,5129841,66.863671,668.636708,18.986943
9,沖縄県,23688,17564,152,1821.0,0.331839,0.057765,1481547,118.551757,1185.517570,122.912064


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      38.266065
std       27.288657
min        7.692183
25%       20.661927
50%       29.603146
75%       50.385253
max      118.551757
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,23688,17564,152,1821.0,0.331839,0.057765,1481547,118.551757,1185.517570,122.912064
4,北海道,504,38530,1138,2573.0,0.248031,-0.057105,5267762,73.143016,731.430160,48.844272
24,愛知県,11592,47882,816,2165.0,-0.059016,0.023445,7575530,63.206139,632.061387,28.578859
44,高知県,19656,1520,20,183.0,0.080000,0.032804,709230,21.431694,214.316935,25.802631
26,東京都,6552,161913,2075,3503.0,0.033970,-0.006695,13834925,117.032076,1170.320764,25.319978
21,広島県,17136,10809,135,644.0,0.372549,-0.098579,2826858,38.236799,382.367986,22.781477
17,岐阜県,10584,8758,166,433.0,0.057143,0.032111,2032490,43.090003,430.900029,21.303918
29,滋賀県,12600,5169,81,271.0,0.137931,-0.023188,1420948,36.377123,363.771229,19.071775
34,福岡県,20160,34300,465,974.0,0.432099,-0.026040,5129841,66.863671,668.636708,18.986943
9,大阪府,13608,100133,2363,1524.0,0.059701,0.019585,8849635,113.149299,1131.492994,17.221049


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    42.000000
mean      0.157716
std       0.430485
min      -0.098579
25%      -0.008829
50%       0.023236
75%       0.122298
max       2.421779
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
6,和歌山県,15120,2629,45,43.0,1.666667,2.421779,954258,27.550201,275.502013,4.506119
41,青森県,1008,2383,30,120.0,-0.062500,1.315065,1275783,18.678725,186.787251,9.405988
39,長崎県,21168,2916,67,74.0,-0.285714,0.625144,1350769,21.587703,215.877030,5.478361
23,愛媛県,19152,2722,72,34.0,-0.200000,0.397372,1369131,19.881224,198.812239,2.483327
11,宮城県,2016,8894,83,117.0,0.000000,0.395505,2292385,38.798020,387.980204,5.103855
12,宮崎県,22680,3030,26,34.0,0.500000,0.316079,1095903,27.648432,276.484324,3.102464
3,兵庫県,14112,39920,1202,697.0,-0.071429,0.268710,5549568,71.933527,719.335271,12.559536
19,岩手県,1512,1465,45,85.0,1.833333,0.217478,1235517,11.857384,118.573844,6.879711
46,鹿児島県,23184,3384,30,181.0,0.280000,0.155581,1630146,20.758877,207.588768,11.103300
2,佐賀県,20664,2513,19,63.0,-0.571429,0.143707,823810,30.504607,305.046066,7.647394


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
21,広島県,17136,10809,135,644.0,0.372549,-0.098579,2826858,38.236799,382.367986,22.781477
42,静岡県,11088,8473,138,346.0,-0.069767,-0.060235,3708556,22.847167,228.471675,9.329777
4,北海道,504,38530,1138,2573.0,0.248031,-0.057105,5267762,73.143016,731.430160,48.844272
18,岡山県,16632,7397,106,286.0,0.850000,-0.049149,1903627,38.857402,388.574022,15.023952
10,奈良県,14616,7809,115,135.0,-0.173913,-0.033222,1353837,57.680504,576.805036,9.971658
34,福岡県,20160,34300,465,974.0,0.432099,-0.026040,5129841,66.863671,668.636708,18.986943
37,群馬県,5040,7861,139,159.0,0.352941,-0.024911,1969439,39.914920,399.149199,8.073365
29,滋賀県,12600,5169,81,271.0,0.137931,-0.023188,1420948,36.377123,363.771229,19.071775
8,大分県,22176,3408,50,139.0,0.294118,-0.020597,1151229,29.603146,296.031459,12.074053
0,三重県,12096,4958,107,183.0,0.045455,-0.013708,1813859,27.333988,273.339879,10.088987


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,10))
plt.subplot(2,2,1) # (rows, columns, panel number)
#plt.xticks(rotation="70",fontsize=9)
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
#plt.bar(all_jp_desc["都道府県名"], all_jp_desc["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")
#plt.title("各地の感染者数", y=0.9)
plt.title("各地の感染者数({})".format(latest_date),y=0.95)
plt.text(5,-1,"※{}時点".format(latest_date),fontsize=9)

plt.subplot(2,2,2) # (rows, columns, panel number)
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])
plt.title("1万人当たりの感染者数({})".format(latest_date),y=0.95)
plt.text(5,-1,"※{}時点".format(latest_date),fontsize=9)

plt.subplot(2,2,3) # (rows, columns, panel number)
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
plt.title("直近1週間の10万人当たりの新規感染者数",y=0.95)
# plt.text(0,chart_data["都道府県名"][-1],"{}時点".format(latest_date))
plt.text(5,-1,"※{}時点".format(latest_date),fontsize=9)

### 新規感染者数の増加速度（直近１週間の前日比平均）
plt.subplot(2,2,4) # (rows, columns, panel number)
chart_data = all_jp.sort_values("直近1週間の新規感染者数前日比平均",na_position="first")
plt.yticks(fontsize=8)
plt.grid(axis="x")
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の新規感染者数前日比平均"])
plt.title("新規感染者数の増加速度（直近１週間の前日比平均）",y=0.95)
plt.text(0,2,"※{}時点".format(latest_date),fontsize=9)

fig.tight_layout()


<IPython.core.display.Javascript object>

In [22]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [23]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  4  2  1  1  0  0  1]
[   466.     20646.875  40827.75   61008.625  81189.5   101370.375
 121551.25  141732.125 161913.   ]


## ヒストグラム

In [24]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([38.,  4.,  3.,  0.,  1.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([11., 23.,  4.,  6.,  0.,  3.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)